In [1]:
import numpy as np

from scipy.fft import dct, idct

from scipy.signal.windows import kaiser_bessel_derived as kbdw

In [2]:
def mdct4(x):
    N = x.shape[0]
    if N%4 != 0:
        raise ValueError("MDCT4 only defined for vectors of length multiple of four.")
    M = N // 2
    N4 = N // 4
    
    rot = np.roll(x, N4)
    rot[:N4] = -rot[:N4]
    t = np.arange(0, N4)
    w = np.exp(-1j*2*np.pi*(t + 1./8.) / N)
    c = np.take(rot,2*t) - np.take(rot, N-2*t-1) \
        - 1j * (np.take(rot, M+2*t) - np.take(rot,M-2*t-1))
    c = (2./np.sqrt(N)) * w * np.fft.fft(0.5 * c * w, N4)
    y = np.zeros(M)
    y[2*t] = np.real(c[t])
    y[M-2*t-1] = -np.imag(c[t])
    return y

def imdct4(x):
    N = x.shape[0]
    if N%2 != 0:
        raise ValueError("iMDCT4 only defined for even-length vectors.")
    M = N // 2
    N2 = N*2
    
    t = np.arange(0,M)
    w = np.exp(-1j*2*np.pi*(t + 1./8.) / N2)
    c = np.take(x,2*t) + 1j * np.take(x,N-2*t-1)
    c = 0.5 * w * c
    c = np.fft.fft(c,M)
    c = ((8 / np.sqrt(N2))*w)*c
    
    rot = np.zeros(N2)
    
    rot[2*t] = np.real(c[t])
    rot[N+2*t] = np.imag(c[t])
    
    t = np.arange(1,N2,2)
    rot[t] = -rot[N2-t-1]
    
    t = np.arange(0,3*M)
    y = np.zeros(N2)
    y[t] = rot[t+M]
    t = np.arange(3*M,N2)
    y[t] = -rot[t-3*M]
    return y

In [3]:
def mdct(x):
    N = x.shape[0]

    if N%4 != 0:
        raise ValueError("MDCT4 only defined for vectors of length multiple of four.")

    N4 = N // 4

    a = x[0*N4:1*N4]
    b = x[1*N4:2*N4]
    c = x[2*N4:3*N4]
    d = x[3*N4:4*N4]

    br = np.flip(b)
    cr = np.flip(c)

    return dct(np.hstack([-cr - d, a - br]), type=4, norm='ortho', orthogonalize=True) / 2


def imdct(y):
    N = y.shape[0] * 2

    if N%4 != 0:
        raise ValueError("IMDCT is only defined for vectors lengths multiple of two.")
    
    N4 = N // 4

    z = idct(y, type=4, norm='ortho', orthogonalize=True)

    z = np.hstack([z, -np.flip(z), -z]) * 2

    return z[N4:5*N//4]

In [4]:
x = np.random.randint(-128, 128, 32)

x = np.array([
        114, 132, 126, 134, 172, 168, 172, 139,
        146, 149, 159, 124, 137, 121, 124, 111, 
        137, 142, 178, 194, 201, 188, 188, 169, 
        198, 204, 223, 213, 205, 218, 209, 193
    ])

# x = x - x.mean()

x1 = x[0:16]
x2 = x[8:24]

zo = x[8:16]

print('x1 = ', x1)
print('x1 = ', x2)
print('zo = ', zo)

r = 1

x1 =  [114 132 126 134 172 168 172 139 146 149 159 124 137 121 124 111]
x1 =  [146 149 159 124 137 121 124 111 137 142 178 194 201 188 188 169]
zo =  [146 149 159 124 137 121 124 111]


In [5]:
kbdf = kbdw(16, np.pi * 0.64)

y1 = mdct4(x1)
y2 = mdct4(x2)

y1[r:] = 0
y2[r:] = 0

z1 = imdct4(y1)
z2 = imdct4(y2)

z = np.rint((z1[8:] + z2[:8])/2).astype(np.int64)

print('zo = ', zo)
print('z  = ', np.rint(z).astype(np.int64))

relerr = abs(z - zo) / zo

print('relerr = ', relerr.round(2))
print('cumrelerr = ', relerr.sum().round(2))

zo =  [146 149 159 124 137 121 124 111]
z  =  [  1  40  77 112 142 166 185 196]
relerr =  [0.99 0.73 0.52 0.1  0.04 0.37 0.49 0.77]
cumrelerr =  4.0


In [6]:
y1 = mdct4(x1 * kbdf)
y2 = mdct4(x2 * kbdf)

y1[r:] = 0
y2[r:] = 0

z1 = imdct4(y1) * kbdf
z2 = imdct4(y2) * kbdf

z = np.rint(z1[8:] + z2[:8]).astype(np.int64)

print('zo = ', zo)
print('z  = ', np.rint(z).astype(np.int64))

relerr = abs(z - zo) / zo

print('relerr = ', relerr.round(2))
print('cumrelerr = ', relerr.sum().round(2))


zo =  [146 149 159 124 137 121 124 111]
z  =  [115 121 129 138 148 159 170 180]
relerr =  [0.21 0.19 0.19 0.11 0.08 0.31 0.37 0.62]
cumrelerr =  2.09


In [7]:
y1 = mdct(x1)
y2 = mdct(x2)

y1[r:] = 0
y2[r:] = 0

z1 = imdct(y1)
z2 = imdct(y2)

z = np.rint((z1[8:] + z2[:8])/2).astype(np.int64)

print('zo = ', zo)
print('z  = ', np.rint(z).astype(np.int64))

relerr = abs(z - zo) / zo

print('relerr = ', relerr.round(2))
print('cumrelerr = ', relerr.sum().round(2))

zo =  [146 149 159 124 137 121 124 111]
z  =  [  1  40  77 112 142 166 185 196]
relerr =  [0.99 0.73 0.52 0.1  0.04 0.37 0.49 0.77]
cumrelerr =  4.0


In [8]:
y1 = mdct(x1 * kbdf)
y2 = mdct(x2 * kbdf)

y1[r:] = 0
y2[r:] = 0

z1 = imdct(y1) * kbdf
z2 = imdct(y2) * kbdf

z = np.rint(z1[8:] + z2[:8]).astype(np.int64)

print('zo = ', zo)
print('z  = ', np.rint(z).astype(np.int64))

relerr = abs(z - zo) / zo

print('relerr = ', relerr.round(2))
print('cumrelerr = ', relerr.sum().round(2))

zo =  [146 149 159 124 137 121 124 111]
z  =  [115 121 129 138 148 159 170 180]
relerr =  [0.21 0.19 0.19 0.11 0.08 0.31 0.37 0.62]
cumrelerr =  2.09
